In [1]:
import numpy as np 
import scipy as sp
import healpy as hp
import matplotlib.pyplot as plt
import mylib
import camb
import pymaster as nmt
from tqdm import tqdm

In [ ]:
nside = 128
lmax = 3*nside - 1
npix = 12 * nside**2

input_cls_lcdm = mylib.get_camb(Alens=1, r=0, lmax=lmax)

input_cls_noBB = np.copy(input_cls_lcdm)
input_cls_noBB[2] = np.zeros(lmax+1)

input_cls_noEE = np.copy(input_cls_lcdm)
input_cls_noEE[1] = np.zeros(lmax+1)
input_cls_noEE[3] = np.zeros(lmax+1)

In [ ]:
mask, weight = mylib.get_weight(nside)
fsky = np.sum(mask)/npix
print(fsky*100)

apo_size = 10
mask_apo = nmt.mask_apodization(weight, apo_size, 'C2')
mask_apo /= mask_apo.max()

R_unapo = sp.sparse.load_npz(f'obsmat_nside{nside}/obsmat.npz')

Z = sp.sparse.diags_array(mask_apo)
ZZ = sp.sparse.block_diag([Z, Z, Z])
R = ZZ @ R_unapo

# pi_b = sp.sparse.load_npz(f'apo{apo_size}/pi_b_5000.npz')

In [ ]:
bin_size = 16
b = nmt.NmtBin.from_nside_linear(nside, bin_size)
ells = b.get_effective_ells()
c2db = ells * (ells+1) /2/np.pi
sigmab = hp.nside2resol(nside)
fwhm = (8*np.log(2))**0.5 * sigmab
bl = hp.gauss_beam(fwhm, lmax)
l = np.arange(lmax+1)
c2d = l * (l+1) / 2/np.pi

In [ ]:
planck_T_noise_props = [25, 50, -1.8]
planck_P_noise_props = [50, 20, -1.4]
planck_T = mylib.get_Nl(planck_T_noise_props, lmax)
planck_P = mylib.get_Nl(planck_P_noise_props, lmax)
planck_noise_cls = np.array([planck_T, planck_P, planck_P, np.zeros(lmax+1)])

lb_T_noise_props = [1.8, 5, -1] 
lb_P_noise_props = [2.16, 5, -1] 
lb_T = mylib.get_Nl(lb_T_noise_props, lmax)
lb_P = mylib.get_Nl(lb_P_noise_props, lmax)
lb_noise_cls = np.array([lb_T, lb_P, lb_P, np.zeros(lmax+1)])

In [ ]:
EE_signal = np.zeros(lmax+1)
EE_noise = np.zeros(lmax+1)
EE_full = np.zeros(lmax+1)
EE_wienered = np.zeros(lmax+1)

for i in range(100):
    signal = hp.synfast(input_cls_noBB, nside, fwhm=fwhm, new=True)
    noise = hp.synfast(planck_noise_cls, nside, new=True)
    # noise = hp.synfast(lb_noise_cls, nside, new=True)

    noisy_E = signal + noise
    wienered_E = mylib.wiener_filter(noisy_E, input_cls_noBB*bl**2, planck_noise_cls)

    EE_signal += hp.anafast(signal)[1]
    EE_noise += hp.anafast(noise)[1]
    EE_full += hp.anafast(noisy_E)[1]
    EE_wienered += hp.anafast(wienered_E)[1]

EE_signal/=100
EE_noise/=100
EE_full/=100
EE_wienered/=100

In [ ]:
plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

plt.plot(c2d*EE_signal/bl**2, label='EE signal ($\Lambda$CDM)')
plt.plot(c2d*EE_noise/bl**2, label='Planck-like EE noise (w/ deg beam)')
plt.plot(c2d*EE_full/bl**2, label='Total EE (signal + noise)')
plt.plot(c2d*EE_wienered/bl**2, label='Wiener filtered total EE')
plt.loglog()

plt.legend(fontsize=10)
plt.xlim([2,2*nside])
plt.ylim([5e-4,5e1])
plt.grid()
plt.ylabel('$D_\ell$ [$\mu$K$^2$]', fontsize=14)
plt.xlabel('$\ell$', fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.savefig('plots/PS_wiener.png', bbox_inches='tight')

In [8]:
# SO_noise = mask_apo * hp.read_map(f'SO_opt/out/{1000}/filterbin_filtered_map.fits', field=None)
# S4_noise = mask_apo * hp.read_map(f'S4/out/{1000}/filterbin_filtered_map.fits', field=None)

In [9]:
# SO_scale = 0.15
# S4_scale = 0.15

# fig, axes = plt.subplots(2,2)

# plt.axes(axes[0][0])
# hp.gnomview(SO_noise[1], cmap='coolwarm', min=-SO_scale, max=SO_scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='SO-like Q noise (filtered)', notext=True, hold=True)

# plt.axes(axes[0][1])
# hp.gnomview(SO_noise[2], cmap='coolwarm', min=-SO_scale, max=SO_scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='SO-like U noise (filtered)', notext=True, hold=True)

# plt.axes(axes[1][0])
# hp.gnomview(S4_noise[1], cmap='coolwarm', min=-S4_scale, max=S4_scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='S4-like Q noise (filtered)', notext=True, hold=True)

# plt.axes(axes[1][1])
# hp.gnomview(S4_noise[2], cmap='coolwarm', min=-S4_scale, max=S4_scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='S4-like U noise (filtered)', notext=True, hold=True)

# plt.savefig('plots/noise.png', bbox_inches='tight')

In [18]:
# pcl = np.load(f'apo{apo_size}/pcl.npy')
# # pcl_KS = np.load(f'apo{apo_size}/pcl_KS.npy')
# # pcl_pure = np.load(f'apo{apo_size}/pcl_pure.npy')

# mask, weight = mylib.get_weight(nside)
# mask_apo_true = nmt.mask_apodization(weight, apo_size, 'C2')
# mask_apo_true /= np.sqrt(np.mean(mask_apo_true**2)) 

# # mll_KS = mylib.get_mll(mask_apo_true, nside, b, pol=True, purify_b=True)[2::4,2::4]
# mll = mylib.get_mll(mask_apo_true, nside, b, pol=True)[2::4,2::4]

# fl = mylib.get_fl(pcl, bl, mll, input_cls_lcdm[2], niter=3)
# # fl_KS = mylib.get_fl(pcl_KS, bl, mll_KS, input_cls_lcdm[2], niter=3)
# # fl_pure = mylib.get_fl(pcl_pure, bl, mll, input_cls_lcdm[2], niter=3)

In [41]:
nmc_noise = 256

nl_SO = np.zeros((nmc_noise, len(ells)))
nl_S4 = np.zeros((nmc_noise, len(ells)))

for i in tqdm(range(nmc_noise)):
    obs_noise_unapo = hp.read_map(f'SO_opt/out/{i+1000}/filterbin_filtered_map.fits', field=None)
    f = nmt.NmtField(mask_apo, mask*obs_noise_unapo[1:], beam=np.sqrt(fl))
    nl_SO[i] = nmt.compute_full_master(f, f, b)[3]
    
    obs_noise_unapo = hp.read_map(f'S4/out/{i+1000}/filterbin_filtered_map.fits', field=None)
    f = nmt.NmtField(mask_apo, mask*obs_noise_unapo[1:], beam=np.sqrt(fl))
    nl_S4[i] = nmt.compute_full_master(f, f, b)[3]

nl_SO_mean = np.mean(nl_SO, axis=0)
nl_SO_std = np.std(nl_SO, axis=0)

nl_S4_mean = np.mean(nl_S4, axis=0)
nl_S4_std = np.std(nl_S4, axis=0)

100%|█████████████████████████████████████████| 256/256 [02:52<00:00,  1.48it/s]


In [42]:
sat_pol_optimal = mylib.get_Nl([np.sqrt(2)*2.1, 50, -3], lmax)
s4 = mylib.get_Nl([1, 50, -3], lmax)

/scratch/yuyang/filterbin/mylib.py:105: RuntimeWarning: divide by zero encountered in power
  Nl = n * (l/knee)**alpha + n


In [64]:
# plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

# plt.errorbar(ells, c2db*nl_SO_mean, yerr=c2db*nl_SO_std, fmt='.', label='Recon. BB of S4-like noise sims')
# plt.errorbar(ells, c2db*nl_S4_mean, yerr=c2db*nl_S4_std, fmt='.', label='Recon. BB of SO-like noise sims')

# plt.plot(c2d*s4, label='S4-like $N_\ell$ (analytic)')
# plt.plot(c2d*sat_pol_optimal, label='SO-like $N_\ell$ (analytic)')

# plt.plot(c2d*input_cls_lcdm[2], label='Lensing BB')

# plt.loglog()
# plt.legend(fontsize=10)
# plt.xlim([40, 2*nside])
# plt.ylim([3e-5,3e-2])
# plt.grid()
# plt.ylabel('$D_\ell$ [$\mu$K$^2$]', fontsize=18)
# plt.xlabel('$\ell$', fontsize=18)
# # plt.title('BB spectra for filtered $\Lambda$CDM maps', fontsize=20, pad=15)

# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)

# plt.savefig('plots/PS_noise.png', bbox_inches='tight')

In [10]:
# np.random.seed(0)
# signal = hp.synfast(input_cls_noBB, nside, fwhm=fwhm, new=True)
# planck_noise = hp.synfast(planck_noise_cls, nside, new=True)

# noisy_E = signal + planck_noise
# wienered_E = mylib.wiener_filter(noisy_E, input_cls_noBB*bl**2, planck_noise_cls)

# noisy_E = mask_apo * hp.alm2map(hp.map2alm(noisy_E)[1], nside)
# wienered_E = mask_apo * hp.alm2map(hp.map2alm(wienered_E)[1], nside)

In [11]:
# scale=3

# fig, axes = plt.subplots(2,1, figsize=(10,10))

# plt.axes(axes[0])
# hp.gnomview(noisy_E, cmap='coolwarm', min=-scale, max=scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='Total E (signal + noise)', notext=True, hold=True)

# plt.axes(axes[1])
# hp.gnomview(wienered_E, cmap='coolwarm', min=-1, max=1, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='Wiener-filtered total E', notext=True, hold=True)
# plt.savefig('plots/wiener.png', bbox_inches='tight')

In [12]:
# np.random.seed(0)
# signal = hp.synfast(input_cls_lcdm, nside, fwhm=fwhm, new=True)
# obs_m = (R @ signal.flatten()).reshape(3, npix)
# masked_m = mask_apo * signal

# scale=1

# fig, axes = plt.subplots(2,2)

# plt.axes(axes[0][0])
# hp.gnomview(masked_m[1], cmap='coolwarm', min=-scale, max=scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='Q', notext=True, hold=True)

# plt.axes(axes[0][1])
# hp.gnomview(masked_m[2], cmap='coolwarm', min=-scale, max=scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='U', notext=True, hold=True)

# plt.axes(axes[1][0])
# hp.gnomview(obs_m[1], cmap='coolwarm', min=-scale, max=scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='Filtered Q', notext=True, hold=True)

# plt.axes(axes[1][1])
# hp.gnomview(obs_m[2], cmap='coolwarm', min=-scale, max=scale, rot=(40,-42), reso=20, ysize=110, unit='$\mu$K', title='Filtered U', notext=True, hold=True)

# plt.savefig('plots/filtered.png', bbox_inches='tight')

In [15]:
# plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

# plt.plot(fl_KS, label='KS purification')
# plt.plot(fl_pure, label='Matrix purification')

# plt.legend(fontsize=10)
# plt.xlim([25, 2*nside])
# plt.ylim([0,1])
# plt.grid()
# plt.ylabel('Power ratio (output/input)', fontsize=14)
# plt.xlabel('$\ell$', fontsize=14)

# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)

# plt.savefig('plots/ft.png', bbox_inches='tight')

In [24]:
# hits = hp.read_map('obsmat_nside128/out/0/filterbin_hits.fits')

In [25]:
# hp.gnomview(hits, cmap='coolwarm', rot=(40,-42), reso=25, ysize=110, unit='hits', title=None, notext=True)
# plt.savefig('plots/hits.png', bbox_inches='tight')

In [14]:
# ten = mylib.get_Nl([10,0,0], 0, lmax)
# twopfive = mylib.get_Nl([2.5,0,0], 0, lmax)
# one = mylib.get_Nl([1,0,0], 0, lmax)

In [15]:
# nmc = 128

# cl_tteebb = np.empty((nmc, len(ells)))
# cl_ttee = np.empty((nmc, len(ells)))

# for i in tqdm(range(nmc)):
#     np.random.seed(i)
#     signal = hp.synfast(input_cls_noBB, nside, fwhm=fwhm, new=True)
#     obs_m_unapo = (R_unapo @ signal.flatten()).reshape(3, npix)
#     f = nmt.NmtField(mask_apo, mask*obs_m_unapo[1:], beam=bl*np.sqrt(fl_KS), purify_b=True)
#     cl_ttee[i] = nmt.compute_full_master(f, f, b)[3]

#     np.random.seed(i)
#     signal = hp.synfast(input_cls_lcdm, nside, fwhm=fwhm, new=True)
#     obs_m_unapo = (R_unapo @ signal.flatten()).reshape(3, npix)
#     f = nmt.NmtField(mask_apo, mask*obs_m_unapo[1:], beam=bl*np.sqrt(fl_KS), purify_b=True)
#     cl_tteebb[i] = nmt.compute_full_master(f, f, b)[3]
    
# ttee_mean = c2db*np.mean(cl_ttee, axis=0)
# ttee_std = c2db*np.std(cl_ttee, axis=0)

# tteebb_mean = c2db*np.mean(cl_tteebb, axis=0)
# tteebb_std =c2db*np.std(cl_tteebb, axis=0)

100%|█████████████████████████████████████████| 128/128 [06:20<00:00,  2.98s/it]


In [16]:
# # plt.figure(figsize=(7, 4))
# plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

# plt.errorbar(ells, ttee_mean, yerr=ttee_std, fmt='o', alpha=0.75, label='$\Lambda$CDM (TTEE) input')
# plt.errorbar(ells, tteebb_mean, yerr=tteebb_std, fmt='o', alpha=0.75, label='$\Lambda$CDM (TTEEBB) input')
# plt.plot(l, c2d*input_cls_lcdm[2], label='Lensing BB')
# # plt.plot(c2d*ttee_mean, label=r'TTEEEE$\rightarrow$BB (leakage)')
# # plt.plot(c2d*ttbb_mean, label=r'TTBB$\rightarrow$BB')
# # plt.plot(c2d*tteebb_mean, label=r'TTEEBB$\rightarrow$BB')

# plt.loglog()
# plt.legend(fontsize=14)
# plt.xlim([20,2*nside])
# plt.ylim([1e-4,1e-1])
# plt.grid()
# plt.ylabel('$D_\ell$ [$\mu$K$^2$]', fontsize=18)
# plt.xlabel('$\ell$', fontsize=18)
# # plt.title('BB spectra for filtered $\Lambda$CDM maps', fontsize=20, pad=15)

# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)

# plt.savefig('plots/leakage.png', bbox_inches='tight')

In [26]:
# # plt.figure(figsize=(7, 4))
# plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

# plt.errorbar(ells, ttee_mean, yerr=ttee_std, fmt='o', alpha=1, label='Filtering leakage ($\Lambda$CDM TTEE)')
# # plt.errorbar(ells, tteebb_mean, yerr=tteebb_std, fmt='o', alpha=0.75, label='$\Lambda$CDM (TTEEBB) input')
# plt.plot(l, c2d*input_cls_lcdm[2], label='Lensing BB')
# plt.plot(l, c2d*ten, ls='dashed',label='10 $\mu$K-arcmin')
# plt.plot(l, c2d*twopfive, ls='dashed',label='2.5 $\mu$K-arcmin')
# plt.plot(l, c2d*one, ls='dashed',label='1 $\mu$K-arcmin')

# plt.loglog()
# plt.legend(fontsize=10)
# plt.xlim([20,2*nside])
# plt.ylim([5e-5,1e-1])
# plt.grid()
# plt.ylabel('$D_\ell$ [$\mu$K$^2$]', fontsize=14)
# plt.xlabel('$\ell$', fontsize=14)
# # plt.title('BB: Filtering leakage vs noise', fontsize=16, pad=15)

# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)

# plt.savefig('plots/leakage_noise.png', bbox_inches='tight')

In [27]:
# nmc = 128

# cl = np.empty((nmc, len(ells)))
# cl_KS = np.empty((nmc, len(ells)))

# for i in tqdm(range(nmc)):
#     np.random.seed(i)
#     masked_signal = mask*hp.synfast(input_cls_lcdm, nside, fwhm=fwhm, new=True)
#     f = nmt.NmtField(mask_apo, masked_signal[1:], beam=bl, purify_b=True)
#     cl_KS[i] = nmt.compute_full_master(f, f, b)[3]

#     np.random.seed(i)
#     masked_signal = mask*hp.synfast(input_cls_lcdm, nside, fwhm=fwhm, new=True)
#     f = nmt.NmtField(mask_apo, masked_signal[1:], beam=bl)
#     cl[i] = nmt.compute_full_master(f, f, b)[3]
    
# cl_mean = c2db*np.mean(cl, axis=0)
# cl_std = c2db*np.std(cl, axis=0)

# cl_KS_mean = c2db*np.mean(cl_KS, axis=0)
# cl_KS_std =c2db*np.std(cl_KS, axis=0)

In [28]:
# plt.style.use(['ggplot', 'seaborn-v0_8-ticks'])

# plt.errorbar(ells, cl_mean, yerr=cl_std, fmt='o', alpha=0.75, label='Pseudo $C_\ell$')
# plt.errorbar(ells, cl_KS_mean, yerr=cl_KS_std, fmt='o', alpha=0.75, label='KS purified')
# plt.plot(l, c2d*input_cls_lcdm[2], label='Lensing BB')

# plt.plot(ells, cl_std, linestyle='dashed', label='Pseudo $C_\ell$ std')
# plt.plot(ells, cl_KS_std, linestyle='dashed', label='KS purified std')

# plt.loglog()
# plt.legend(fontsize=10)
# plt.xlim([20,2*nside])
# plt.ylim([5e-5,5e-2])
# plt.grid()
# plt.ylabel('$D_\ell$ [$\mu$K$^2$]', fontsize=14)
# plt.xlabel('$\ell$', fontsize=14)
# # plt.title('BB spectra for masked $\Lambda$CDM maps (fsky=4.5%)', fontsize=16, pad=15)

# plt.xticks(fontsize=12)
# plt.yticks(fontsize=12)

# plt.savefig('plots/KS.png', bbox_inches='tight')